In [ ]:
import glob 
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla
import random
import time
import threading
import queue
import numpy as np

Connect to the server and initialize synchronous mode

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

world = client.get_world()

In [ ]:
# Create a synchronous mode context
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1  # Simulation step size in seconds
world.apply_settings(settings)

In [ ]:
for actor in world.get_actors():
    if actor.type_id[:7] == "vehicle":
        ego_vehicle = actor

Spawn a Second vehicle and set it to autopilot mode

In [ ]:
bp = random.choice(world.get_blueprint_library().filter('*vehicle*'))
transform = random.choice(world.get_map().get_spawn_points())
second_vehicle = world.try_spawn_actor(bp, transform)
second_vehicle.set_autopilot(True)

Does imitating second vehicle's behavior work? No

In [ ]:
while True:
    ego_vehicle.apply_control(second_vehicle.get_control())

Does using `VehiclePIDController` class work? No

Reference: agents/navigation/controller.py

In [ ]:
from agents.navigation.controller import VehiclePIDController

args = dict(K_P=1, K_I=0, K_D=0)
controller = VehiclePIDController(ego_vehicle,args_lateral=args,args_longitudinal=args)
map = world.get_map()

In [ ]:
while True:
    waypoint = map.get_waypoint(second_vehicle.get_location(),project_to_road=True)

    speed = second_vehicle.get_velocity()
    target_speed = (speed.x**2 + speed.y**2 + speed.z**2)**0.5

    controller.run_step(waypoint=waypoint,target_speed=target_speed)

Does using *agents* work? **Yes**

Reference: https://carla.readthedocs.io/en/0.9.14/adv_agents/

In [ ]:
from agents.navigation.behavior_agent import BehaviorAgent

agent = BehaviorAgent(ego_vehicle, behavior='aggressive')

In [ ]:
agent.ignore_traffic_lights(True)
agent.set_destination(second_vehicle.get_location())
# Follow second vehicle
while True:
    if agent.done():
        agent.set_destination(second_vehicle.get_location())

    ego_vehicle.apply_control(agent.run_step())

To store desired data in seperate files in the *out* folder

In [ ]:
def process_data(queue_data, output_folder):
    while True:
        try:
            # Wait for 1 second for data to be available in the queue
            data = queue_data.get(timeout=1.0)  

            # Create a filename using the timestamp of the data
            timestamp = str(int(time.time() * 1000))  # Milliseconds since epoch
            filename = os.path.join(output_folder, f'data_{timestamp}.txt')

            # Write the data to the file
            with open(filename, 'w') as f:
                f.write(str(data))

        except queue.Empty:
            continue

In [ ]:
output_folder = '/home/parsa/Cheetah/carla_simulator/out'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
# Set up the data processing thread
data_queue = queue.Queue()
data_thread = threading.Thread(target=process_data, args=(data_queue, output_folder))
data_thread.start()

In [ ]:
record_time = 60 # Seconds
time0 = time.time()
while True:  
    # Put the data into the queue
    data = {
        'vehicle_location': (ego_vehicle.get_location().x,ego_vehicle.get_location().y,ego_vehicle.get_location().z),
    }
    data_queue.put(data)

    time1 = time.time()
    if (time1-time0) > record_time:
        break

Attempt to draw boxes around vehicles

https://github.com/carla-simulator/carla/issues/1212    (didn't work - should be debugged)

In [ ]:
crashed = False
while not crashed:
    world.wait_for_tick()
    for vehicle in world.get_actors().filter("vehicle*"):
        transform = vehicle.get_transform()
        bounding_box = vehicle.bounding_box
        bounding_box.location += transform.location
        world.debug.draw_box(bounding_box, transform.rotation)
        if vehicle != ego_vehicle:
            distance = ego_vehicle.get_location().distance(vehicle.get_location())
            world.debug.draw_string(vehicle.get_location(), f'{distance:.1f} m')

In [ ]:
crashed = False
while not crashed:
    for vehicle in world.get_actors().filter("vehicle*"):
        if vehicle != ego_vehicle:
            distance = ego_vehicle.get_location().distance(vehicle.get_location())
            if distance < 100:
                loc = vehicle.get_location()
                loc.z += 4
                world.debug.draw_string(loc, f'{distance:.1f} m')

Reference: https://carla.readthedocs.io/en/latest/tuto_G_bounding_boxes/  (worked properly)

In [ ]:
import carla
import math
import random
import time
import queue
import numpy as np
import cv2

client = carla.Client('localhost', 2000)
world  = client.get_world()
bp_lib = world.get_blueprint_library()

# spawn vehicle
vehicle_bp =bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(world.get_map().get_spawn_points()))

# spawn camera
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
vehicle.set_autopilot(True)

# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

for i in range(10):
    bp = random.choice(world.get_blueprint_library().filter('*vehicle*'))
    transform = random.choice(world.get_map().get_spawn_points())
    vv = world.try_spawn_actor(bp, transform)
    vv.set_autopilot(True)

# Create a queue to store and retrieve the sensor data
image_queue = queue.Queue()
camera.listen(image_queue.put)

edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]


In [ ]:
def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]

In [ ]:
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

3D boxes around vehicles

In [ ]:
# 3D
while True:
    # Retrieve and reshape the image
    world.tick()
    image = image_queue.get()

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

    # Get the camera matrix 
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
    # Get the attributes from the camera
    image_w = camera_bp.get_attribute("image_size_x").as_int()
    image_h = camera_bp.get_attribute("image_size_y").as_int()
    fov = camera_bp.get_attribute("fov").as_float()

    # Calculate the camera projection matrix to project from 3D -> 2D
    K = build_projection_matrix(image_w, image_h, fov)

    for npc in world.get_actors().filter('*vehicle*'):

        # Filter out the ego vehicle
        if npc.id != vehicle.id:

            bb = npc.bounding_box
            dist = npc.get_transform().location.distance(vehicle.get_transform().location)

            # Filter for the vehicles within 50m
            if dist < 50:

            # Calculate the dot product between the forward vector
            # of the vehicle and the vector between the vehicle
            # and the other vehicle. We threshold this dot product
            # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
                forward_vec = vehicle.get_transform().get_forward_vector()
                ray = npc.get_transform().location - vehicle.get_transform().location

                if forward_vec.dot(ray) > 1:
                    p1 = get_image_point(bb.location, K, world_2_camera)
                    verts = [v for v in bb.get_world_vertices(npc.get_transform())]
                    for edge in edges:
                        p1 = get_image_point(verts[edge[0]], K, world_2_camera)
                        p2 = get_image_point(verts[edge[1]],  K, world_2_camera)
                        cv2.line(img, (int(p1[0]),int(p1[1])), (int(p2[0]),int(p2[1])), (255,0,0, 255), 1)        

    cv2.imshow('ImageWindowName',img)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

2D boxes around vehicles

In [ ]:
# 2D
while True:

    # Retrieve and reshape the image
    world.tick()
    image = image_queue.get()

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

    # Get the camera matrix 
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
    # Get the attributes from the camera
    image_w = camera_bp.get_attribute("image_size_x").as_int()
    image_h = camera_bp.get_attribute("image_size_y").as_int()
    fov = camera_bp.get_attribute("fov").as_float()

    # Calculate the camera projection matrix to project from 3D -> 2D
    K = build_projection_matrix(image_w, image_h, fov)

    for npc in world.get_actors().filter('*vehicle*'):

        # Filter out the ego vehicle
        if npc.id != vehicle.id:

            bb = npc.bounding_box
            dist = npc.get_transform().location.distance(vehicle.get_transform().location)

            # Filter for the vehicles within 50m
            if dist < 50:

            # Calculate the dot product between the forward vector
            # of the vehicle and the vector between the vehicle
            # and the other vehicle. We threshold this dot product
            # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
                forward_vec = vehicle.get_transform().get_forward_vector()
                ray = npc.get_transform().location - vehicle.get_transform().location

                if forward_vec.dot(ray) > 1:
                    p1 = get_image_point(bb.location, K, world_2_camera)
                    verts = [v for v in bb.get_world_vertices(npc.get_transform())]
                    x_max = -10000
                    x_min = 10000
                    y_max = -10000
                    y_min = 10000

                    for vert in verts:
                        p = get_image_point(vert, K, world_2_camera)
                        # Find the rightmost vertex
                        if p[0] > x_max:
                            x_max = p[0]
                        # Find the leftmost vertex
                        if p[0] < x_min:
                            x_min = p[0]
                        # Find the highest vertex
                        if p[1] > y_max:
                            y_max = p[1]
                        # Find the lowest  vertex
                        if p[1] < y_min:
                            y_min = p[1]

                    cv2.line(img, (int(x_min),int(y_min)), (int(x_max),int(y_min)), (0,0,255, 255), 1)
                    cv2.line(img, (int(x_min),int(y_max)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
                    cv2.line(img, (int(x_min),int(y_min)), (int(x_min),int(y_max)), (0,0,255, 255), 1)
                    cv2.line(img, (int(x_max),int(y_min)), (int(x_max),int(y_max)), (0,0,255, 255), 1)


    cv2.imshow('ImageWindowName',img)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

For camera calibration, vehicle sizing procedure

In [ ]:
import carla
import math
import random
import time
import queue
import numpy as np
import cv2

client = carla.Client('localhost', 2000)
world  = client.get_world()
bp_lib = world.get_blueprint_library()

for actor in world.get_actors():
    if actor.type_id[:7] == "vehicle":
        vehicle = actor

In [ ]:
bounding_box = ego_vehicle.bounding_box
verts = [v for v in bounding_box.get_world_vertices(ego_vehicle.get_transform())]
for v in verts:
    print(v)

Extract Local speed of vehicle 

In [ ]:
import carla
import random
import time
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(2.0)
world = client.get_world()

# Create a synchronous mode context
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1  # Simulation step size in seconds
world.apply_settings(settings)

In [ ]:
ego_vehicle = None
while ego_vehicle is None:
    print("Waiting for the ego vehicle...")
    time.sleep(1)
    possible_vehicles = world.get_actors().filter('vehicle.*')
    for vehicle in possible_vehicles:
        if vehicle.attributes['role_name'] == 'hero':
            print("Ego vehicle found")
            ego_vehicle = vehicle
            break

In [ ]:
bp = random.choice(world.get_blueprint_library().filter('*vehicle*'))
transform = random.choice(world.get_map().get_spawn_points())
ego_vehicle = world.try_spawn_actor(bp, transform)
if ego_vehicle != None:
    ego_vehicle.set_autopilot(True)
    print(ego_vehicle.type_id)

In [ ]:
vehicles = []
for i in range(20):
    bp = random.choice(world.get_blueprint_library().filter('*vehicle*'))
    transform = random.choice(world.get_map().get_spawn_points())
    tmp_vehicle = world.try_spawn_actor(bp, transform)
    if tmp_vehicle != None:
        tmp_vehicle.set_autopilot(True)
        vehicles.append(tmp_vehicle)


In [ ]:
#############
def get_vehicle_lon_speed(ego_vehicle: carla.Vehicle, second_vehicle: carla.Vehicle):
        """
                Get the longitudinal speed of a carla vehicle
                :param carla_vehicle: the carla vehicle
                :type carla_vehicle: carla.Vehicle
                :return: speed of a carla vehicle [m/s]
                :rtype: float64
        """
        ego_velocity = ego_vehicle.get_velocity()
        second_velocity = second_vehicle.get_velocity()
        vec4 = np.array([second_velocity.x - ego_velocity.x,
                        second_velocity.y - ego_velocity.y,
                        second_velocity.z - ego_velocity.z,
                        1]).reshape(4, 1)
        carla_trans = np.array(ego_vehicle.get_transform().get_matrix())
        carla_trans.reshape(4, 4)
        carla_trans[0:3, 3] = 0.0
        vel_in_vehicle = np.linalg.inv(carla_trans) @ vec4
        return vel_in_vehicle[0]

In [ ]:
#############
while True:
    world.tick()
    for vehicle in vehicles:
        rel_vel = get_vehicle_lon_speed(ego_vehicle, vehicle)
        dist = ego_vehicle.get_location().distance(vehicle.get_location())
        if (abs(dist)/rel_vel < 2):
            print(f"-------- Collision with {vehicle.type_id}--------")
        else:
            print("-------- OK --------")

In [ ]:
#$$$$$$$$$$$$$$$$$$$$$$$$$$$
def get_vehicle_lon_speed(ego_vehicle: carla.Vehicle):
        """
                Get the longitudinal speed of a carla vehicle
                :param carla_vehicle: the carla vehicle
                :type carla_vehicle: carla.Vehicle
                :return: speed of a carla vehicle [m/s]
                :rtype: float64
        """
        ego_velocity = ego_vehicle.get_velocity()
        vec4 = np.array([ego_velocity.x,
                        ego_velocity.y,
                        ego_velocity.z,
                        1]).reshape(4, 1)
        carla_trans = np.array(ego_vehicle.get_transform().get_matrix())
        carla_trans.reshape(4, 4)
        carla_trans[0:3, 3] = 0.0
        vel_in_vehicle = np.linalg.inv(carla_trans) @ vec4
        return vel_in_vehicle[0]

In [ ]:
#$$$$$$$$$$$$$$$$$$$$$$$$$$$
spectator = world.get_spectator()
while True:
    world.tick()
    transform = ego_vehicle.get_transform()
    spectator.set_transform(carla.Transform(transform.location + carla.Location(z=30),carla.Rotation(pitch=-90)))
    time.sleep(1)
    for vehicle in vehicles:
        ego_vel = get_vehicle_lon_speed(ego_vehicle)
        veh_vel = get_vehicle_lon_speed(vehicle)
        rel_vel = veh_vel - ego_vel
        dist = ego_vehicle.get_location().distance(vehicle.get_location())
        if (abs(dist)/rel_vel < 2):
            print(f"-------- Collision with {vehicle.type_id}--------")
        else:
            print("-------- OK --------")
    print("", flush=True)